#Service-Oriented Architecture (SOA)

The fundamental concept of breaking down software systems into modular services that are independent, loosely coupled, and interoperable. Services communicate with each other over a network using standard protocols.

Producer

In [ ]:
from flask import Flask

app = Flask(__name__)

@app.route('/greet', methods=['GET'])
def greet():
    return "Hello from Service A!"

if __name__ == '__main__':
    app.run(host='0.0.0.0', port=5000)

Consumer

In [ ]:
import requests

def consume_greeting():
    response = requests.get('http://localhost:5000/greet')
    if response.status_code == 200:
        return response.text
    else:
        return "Failed to get greeting from Service A"

if __name__ == '__main__':
    greeting = consume_greeting()
    print("Received greeting:", greeting)

#Service-Inventory

An inventory of discoverable services that are available for use within the architecture. This pattern emphasizes the importance of cataloging and managing services centrally.

Service-Registry

In [ ]:
from flask import Flask, jsonify, request

app = Flask(__name__)
services = {}

@app.route('/register', methods=['POST'])
def register_service():
    data = request.get_json()
    service_name = data.get('name')
    service_url = data.get('url')

    if service_name and service_url:
        services[service_name] = service_url
        return jsonify({'message': f'Service {service_name} registered successfully'}), 201
    else:
        return jsonify({'error': 'Invalid service registration data'}), 400

@app.route('/discover/<service_name>', methods=['GET'])
def discover_service(service_name):
    if service_name in services:
        return jsonify({'service_url': services[service_name]}), 200
    else:
        return jsonify({'error': f'Service {service_name} not found'}), 404

if __name__ == '__main__':
    app.run(host='0.0.0.0', port=5001)

Registering a Service


In [ ]:
import requests

service_name = 'ServiceA'
service_url = 'http://localhost:5000'  # URL of the service to register

payload = {
    'name': service_name,
    'url': service_url
}

response = requests.post('http://localhost:5001/register', json=payload)

if response.status_code == 201:
    print(f"Service {service_name} registered successfully.")
else:
    print(f"Failed to register Service {service_name}.")

Discovering a Service


In [ ]:
import requests

service_name = 'ServiceA'  # Name of the service to discover

response = requests.get(f'http://localhost:5001/discover/{service_name}')

if response.status_code == 200:
    service_info = response.json()
    service_url = service_info['service_url']
    print(f"Service {service_name} found at {service_url}.")
else:
    print(f"Service {service_name} not found.")

#Service-Oriented Integration (SOI):
Integrating various services and systems using standardized interfaces and protocols. SOI promotes the reuse of existing services and data sources to build new applications.

In [ ]:
from flask import Flask, jsonify

app = Flask(__name__)

users = {
    '1': {'name': 'Alice', 'age': 30, 'city': 'New York'},
    '2': {'name': 'Bob', 'age': 25, 'city': 'San Francisco'},
}

@app.route('/users/<user_id>', methods=['GET'])
def get_user(user_id):
    user = users.get(user_id)
    if user:
        return jsonify(user), 200
    else:
        return jsonify({'error': 'User not found'}), 404

if __name__ == '__main__':
    app.run(host='0.0.0.0', port=5000)

In [ ]:
from flask import Flask, jsonify

app = Flask(__name__)

products = {
    '101': {'name': 'Laptop', 'price': 1200},
    '102': {'name': 'Smartphone', 'price': 800},
}

@app.route('/products/<product_id>', methods=['GET'])
def get_product(product_id):
    product = products.get(product_id)
    if product:
        return jsonify(product), 200
    else:
        return jsonify({'error': 'Product not found'}), 404

if __name__ == '__main__':
    app.run(host='0.0.0.0', port=5001)

Integration Service

In [ ]:
from flask import Flask, jsonify
import requests

app = Flask(__name__)

service_a_url = 'http://localhost:5000'
service_b_url = 'http://localhost:5001'

@app.route('/user_product/<user_id>', methods=['GET'])
def get_user_product(user_id):
    user_url = f"{service_a_url}/users/{user_id}"
    product_url = f"{service_b_url}/products/{user_id}"

    user_response = requests.get(user_url)
    product_response = requests.get(product_url)

    if user_response.status_code == 200 and product_response.status_code == 200:
        user_data = user_response.json()
        product_data = product_response.json()

        user_product_info = {
            'user_id': user_id,
            'user_name': user_data['name'],
            'user_age': user_data['age'],
            'product_name': product_data['name'],
            'product_price': product_data['price']
        }

        return jsonify(user_product_info), 200
    else:
        return jsonify({'error': 'Failed to retrieve user or product data'}), 404

if __name__ == '__main__':
    app.run(host='0.0.0.0', port=5002)

#Enterprise Service Bus (ESB):
A centralized middleware component that facilitates communication and integration between services using a bus-like infrastructure. The ESB manages message routing, transformation, and mediation.

ESB (Message Broker)


In [ ]:
import pika

# Establish connection to RabbitMQ server
connection = pika.BlockingConnection(pika.ConnectionParameters('localhost'))
channel = connection.channel()

# Declare the exchange
channel.exchange_declare(exchange='esb_exchange', exchange_type='direct')

# Declare queues for services
channel.queue_declare(queue='service_a_queue')
channel.queue_declare(queue='service_b_queue')

def callback(ch, method, properties, body):
    # Handle received message
    message = body.decode()
    print(f"Received message: {message}")

    # Example: Route message based on routing key
    if method.routing_key == 'service_a':
        # Forward message to ServiceA queue
        channel.basic_publish(exchange='', routing_key='service_a_queue', body=message)
        print("Message forwarded to ServiceA")
    elif method.routing_key == 'service_b':
        # Forward message to ServiceB queue
        channel.basic_publish(exchange='', routing_key='service_b_queue', body=message)
        print("Message forwarded to ServiceB")

# Bind queues to exchange with routing keys
channel.queue_bind(exchange='esb_exchange', queue='service_a_queue', routing_key='service_a')
channel.queue_bind(exchange='esb_exchange', queue='service_b_queue', routing_key='service_b')

# Start consuming messages
channel.basic_consume(queue='esb_queue', on_message_callback=callback, auto_ack=True)

print('ESB (Message Broker) started. Waiting for messages...')
channel.start_consuming()

ServiceA

In [ ]:
import pika

# Establish connection to RabbitMQ server
connection = pika.BlockingConnection(pika.ConnectionParameters('localhost'))
channel = connection.channel()

# Publish message to ESB with routing key for ServiceA
channel.basic_publish(exchange='esb_exchange', routing_key='service_a', body='Hello from ServiceA')

print("Message sent from ServiceA to ESB")

connection.close()

ServiceB


In [ ]:
import pika

# Establish connection to RabbitMQ server
connection = pika.BlockingConnection(pika.ConnectionParameters('localhost'))
channel = connection.channel()

# Publish message to ESB with routing key for ServiceB
channel.basic_publish(exchange='esb_exchange', routing_key='service_b', body='Data from ServiceB')

print("Message sent from ServiceB to ESB")

connection.close()

#Service Choreography:
Describing the interaction and coordination of services in a distributed system. Service choreography focuses on defining the sequence of interactions without a central controller.

In [ ]:
import time
from threading import Thread, Event

class ServiceA(Thread):
    def __init__(self):
        super().__init__()
        self.stop_event = Event()

    def run(self):
        print("Service A started.")
        while not self.stop_event.is_set():
            # Perform service-specific tasks
            time.sleep(2)  # Simulate processing time
            print("Service A: Sending event to Service B")
            event = {
                'event_type': 'GREETING',
                'data': 'Hello from Service A!'
            }
            service_b.handle_event(event)

    def stop(self):
        self.stop_event.set()

service_a = ServiceA()
service_a.start()

Service A started.


In [ ]:
class ServiceB:
    def handle_event(self, event):
        event_type = event['event_type']
        if event_type == 'GREETING':
            data = event['data']
            print(f"Service B: Received greeting - {data}")

service_b = ServiceB()

In [ ]:
import time
from threading import Thread, Event
from queue import Queue

class ServiceA(Thread):
    def __init__(self, event_queue):
        super().__init__()
        self.event_queue = event_queue
        self.stop_event = Event()

    def run(self):
        print("Service A started.")
        while not self.stop_event.is_set():
            time.sleep(2)  # Simulate processing time
            event = {'event_type': 'GREETING', 'data': 'Hello from Service A!'}
            self.event_queue.put(event)

    def stop(self):
        self.stop_event.set()

# Create an event queue for inter-service communication
event_queue = Queue()
service_a = ServiceA(event_queue)
service_a.start()

Service A started.


In [ ]:
import time
from threading import Thread
from queue import Queue

class ServiceB(Thread):
    def __init__(self, event_queue):
        super().__init__()
        self.event_queue = event_queue
        self.stop_event = Event()

    def run(self):
        print("Service B started.")
        while not self.stop_event.is_set():
            if not self.event_queue.empty():
                event = self.event_queue.get()
                self.handle_event(event)
            time.sleep(1)  # Simulate processing time

    def handle_event(self, event):
        event_type = event['event_type']
        if event_type == 'GREETING':
            data = event['data']
            print(f"Service B received: {data}")

# Create an event queue for inter-service communication
event_queue = Queue()
service_b = ServiceB(event_queue)
service_b.start()

Service B started.


In [ ]:
import asyncio
import websockets

async def send_messages():
    async with websockets.connect('ws://localhost:8765') as websocket:
        while True:
            message = input("Enter a message to send to ServiceB (or type 'exit' to quit): ")
            if message.lower() == 'exit':
                break
            await websocket.send(message)
            print(f"Sent to ServiceB: {message}")

asyncio.run(send_messages())

In [ ]:
import asyncio
import websockets

async def handle_messages(websocket, path):
    print("ServiceB connected to ServiceA")
    try:
        async for message in websocket:
            print(f"Received from ServiceA: {message}")
            # Process the message here (e.g., perform some action based on the message)
    finally:
        print("ServiceA disconnected")

start_server = websockets.serve(handle_messages, 'localhost', 8765)

asyncio.get_event_loop().run_until_complete(start_server)
asyncio.get_event_loop().run_forever()

#Service Orchestration:
Coordination of multiple services to achieve a specific business goal. In this pattern, a central orchestrator (or process manager) controls the flow of messages and coordinates service invocations.

ServiceOrchestrator

In [ ]:
from flask import Flask, jsonify
import requests

app = Flask(__name__)

@app.route('/orchestrate', methods=['GET'])
def orchestrate_workflow():
    try:
        # Step 1: Invoke ServiceA
        response_a = requests.get('http://localhost:5000/greet')
        if response_a.status_code == 200:
            greeting_message = response_a.text.strip()
        else:
            raise Exception("Failed to get greeting from ServiceA")

        # Step 2: Invoke ServiceB with data from ServiceA
        payload = {'message': greeting_message}
        response_b = requests.post('http://localhost:5001/process', json=payload)
        if response_b.status_code == 200:
            processed_data = response_b.json()
        else:
            raise Exception("Failed to process data in ServiceB")

        # Step 3: Invoke ServiceC with processed data from ServiceB
        response_c = requests.post('http://localhost:5002/analyze', json=processed_data)
        if response_c.status_code == 200:
            analysis_result = response_c.json()
            return jsonify(analysis_result), 200
        else:
            raise Exception("Failed to analyze data in ServiceC")

    except Exception as e:
        error_message = str(e)
        return jsonify({'error': error_message}), 500

if __name__ == '__main__':
    app.run(host='0.0.0.0', port=5003)

#Composite Services:
Aggregating multiple services into a single logical unit to provide more complex functionality. Composite services orchestrate lower-level services to fulfill specific business requirements.

In [ ]:
from flask import Flask, jsonify
import requests

app = Flask(__name__)

@app.route('/composite_function', methods=['GET'])
def composite_function():
    try:
        # Step 1: Invoke ServiceA to get a greeting message
        response_a = requests.get('http://localhost:5000/greet')
        if response_a.status_code == 200:
            greeting_message = response_a.text.strip()
        else:
            raise Exception("Failed to get greeting from ServiceA")

        # Step 2: Invoke ServiceB to get some data
        response_b = requests.get('http://localhost:5001/data')
        if response_b.status_code == 200:
            data = response_b.json()
        else:
            raise Exception("Failed to get data from ServiceB")

        # Step 3: Combine results from ServiceA and ServiceB
        composite_result = {
            'greeting': greeting_message,
            'data_from_service_b': data
        }

        return jsonify(composite_result), 200

    except Exception as e:
        error_message = str(e)
        return jsonify({'error': error_message}), 500

if __name__ == '__main__':
    app.run(host='0.0.0.0', port=5004)

#Service Registry and Discovery:
A pattern that involves a central repository (registry) where services are published and can be discovered by other services dynamically at runtime. This facilitates loose coupling and runtime flexibility.

ServiceRegistry

In [ ]:
from flask import Flask, jsonify, request

app = Flask(__name__)
services = {}

@app.route('/register', methods=['POST'])
def register_service():
    data = request.get_json()
    service_name = data.get('name')
    service_url = data.get('url')

    if service_name and service_url:
        services[service_name] = service_url
        return jsonify({'message': f'Service {service_name} registered successfully'}), 201
    else:
        return jsonify({'error': 'Invalid service registration data'}), 400

@app.route('/discover/<service_name>', methods=['GET'])
def discover_service(service_name):
    if service_name in services:
        return jsonify({'service_url': services[service_name]}), 200
    else:
        return jsonify({'error': f'Service {service_name} not found'}), 404

if __name__ == '__main__':
    app.run(host='0.0.0.0', port=5000)

ServiceA

In [ ]:
from flask import Flask

app = Flask(__name__)

@app.route('/greet', methods=['GET'])
def greet():
    return "Hello from ServiceA!"

if __name__ == '__main__':
    app.run(host='0.0.0.0', port=5001)

Service Registry Client


In [ ]:
import requests

def register_service(name, url):
    data = {'name': name, 'url': url}
    response = requests.post('http://localhost:5000/register', json=data)
    return response.json()

def discover_service(service_name):
    response = requests.get(f'http://localhost:5000/discover/{service_name}')
    if response.status_code == 200:
        return response.json()['service_url']
    else:
        return None

# Register ServiceA with the service registry
register_service('ServiceA', 'http://localhost:5001')

# Discover ServiceA using the service registry
service_a_url = discover_service('ServiceA')
if service_a_url:
    print(f"ServiceA discovered at: {service_a_url}")
else:
    print("ServiceA not found in the registry")

#Policy-Driven SOA:
Implementing policies (e.g., security, governance, quality of service) at the service level. Policies are centrally managed and enforced across the architecture to ensure compliance and consistency.

In [ ]:
import time
from functools import wraps

# Define a decorator function for rate limiting
def rate_limit(max_requests, period):
    """
    Decorator function to enforce rate limiting on a service endpoint.

    Args:
        max_requests (int): Maximum number of requests allowed in the specified period.
        period (float): Time period (in seconds) during which the requests are counted.

    Returns:
        function: Decorated function with rate-limiting enforcement.
    """
    def decorator(func):
        request_history = []

        @wraps(func)
        def wrapper(*args, **kwargs):
            current_time = time.time()

            # Remove expired entries from request history
            request_history[:] = [req_time for req_time in request_history if (current_time - req_time) <= period]

            if len(request_history) < max_requests:
                # Add current request time to history
                request_history.append(current_time)
                return func(*args, **kwargs)
            else:
                return "Rate limit exceeded. Please try again later.", 429  # HTTP 429 Too Many Requests

        return wrapper
    return decorator

In [ ]:
from flask import Flask
from policy_enforcement import rate_limit

app = Flask(__name__)

# Apply rate limiting policy (max 5 requests per 10 seconds) to the /example endpoint
@app.route('/example')
@rate_limit(max_requests=5, period=10)
def example_endpoint():
    return "Hello, welcome to the example endpoint!"

if __name__ == '__main__':
    app.run(host='0.0.0.0', port=5000)

#Cloud-Based SOA:
Leveraging cloud computing resources to implement and deploy services within an SOA. This pattern enables scalability, elasticity, and accessibility of services across distributed environments.

Create AWS Lambda Function

In [ ]:
def lambda_handler(event, context):
    # Process incoming event (e.g., perform some computation)
    message = event.get('message', 'Hello from AWS Lambda!')
    return {
        'statusCode': 200,
        'body': message
    }


Deploy AWS Lambda Function

In [ ]:
# Create a deployment package (ZIP file) containing the lambda_function.py
zip lambda_function.zip lambda_function.py

# Create the Lambda function using AWS CLI
aws lambda create-function \
    --function-name my-lambda-function \
    --runtime python3.8 \
    --handler lambda_function.lambda_handler \
    --role <lambda-execution-role-arn> \
    --zip-file fileb://lambda_function.zip

Invoke AWS Lambda Function

In [ ]:
aws lambda invoke \
    --function-name my-lambda-function \
    --payload '{"message": "Hello from AWS Lambda!"}' \
    output.json

Integrate with AWS API Gateway

In [ ]:
# Create an API Gateway REST API
aws apigateway create-rest-api --name my-api

# Get the API ID
API_ID=$(aws apigateway get-rest-apis --query "items[?name=='my-api'].id" --output text)

# Create a resource and method
aws apigateway create-resource --rest-api-id $API_ID --parent-id $PARENT_ID --path-part myresource
aws apigateway put-method --rest-api-id $API_ID --resource-id $RESOURCE_ID --http-method GET --authorization-type "NONE" --request-parameters method.request.querystring.message=true
aws apigateway put-integration --rest-api-id $API_ID --resource-id $RESOURCE_ID --http-method GET --type AWS_PROXY --integration-http-method POST --uri arn:aws:apigateway:us-east-1:lambda:path/2015-03-31/functions/arn:aws:lambda:us-east-1:123456789012:function:my-lambda-function/invocations
aws apigateway create-deployment --rest-api-id $API_ID --stage-name test

# Invoke the API endpoint
curl -X GET https://$API_ID.execute-api.us-east-1.amazonaws.com/test/myresource?message=Hello